 # Home Credit Default Risk - Data Processing and Feature Engineering

 It's important to note that most features are derived from domain knowledge and experimentation.

 ## Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMRegressor
import gc
import warnings

warnings.filterwarnings('ignore')


 ## Configuration

In [100]:
INPUT_DIR = r'C:\Users\kresi\OneDrive\Desktop\Turing college\Project11xx\data/'
APPLICATION_TRAIN = INPUT_DIR + 'application_train.csv'
APPLICATION_TEST = INPUT_DIR + 'application_test.csv'
BUREAU = INPUT_DIR + 'bureau.csv'
BUREAU_BALANCE = INPUT_DIR + 'bureau_balance.csv'
PREVIOUS_APPLICATION = INPUT_DIR + 'previous_application.csv'
POS_CASH = INPUT_DIR + 'POS_CASH_balance.csv'
INSTALLMENTS_PAYMENTS = INPUT_DIR + 'installments_payments.csv'
CREDIT_CARD_BALANCE = INPUT_DIR + 'credit_card_balance.csv'

OUTPUT_DIR = r'C:\Users\kresi\OneDrive\Desktop\Turing college\Project11xx/'
PROCESSED_TRAIN = OUTPUT_DIR + 'processed_train.csv'
PROCESSED_TEST = OUTPUT_DIR + 'processed_test.csv'
TRAIN_TARGET = OUTPUT_DIR + 'train_target.csv'


 ## Helper Functions

 Helper functions:
- `read_csv`: Reads CSV files
- `reduce_mem_usage`: Optimizes memory usage by downcasting numeric columns.

In [101]:
def read_csv(file_path, nrows=None):
    """Read CSV file using pandas."""
    return pd.read_csv(file_path, nrows=nrows)

def reduce_mem_usage(df):
    """Reduce memory usage of a dataframe by downcasting numeric columns."""
    for col in df.columns:
        if df[col].dtype != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(df[col].dtype)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    return df


 ## Process Application Data

 The `process_application` function handles the main application data:

1. Reads and reduces memory usage of the dataframe.
2. Handles missing and erroneous values.
3. Removes unnecessary document flag features.
4. Converts days to years for certain features.
5. Fills categorical columns with 'XNA' for missing values.
6. Performs extensive feature engineering, creating new features based on existing ones.
7. Encodes categorical variables using LabelEncoder.

In [102]:
def process_application(file_path, is_train=True):
    """Process application_{train|test}.csv"""
    df = read_csv(file_path)
    df = reduce_mem_usage(df)

    df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].replace(365243, np.nan)
    df['DAYS_LAST_PHONE_CHANGE'] = df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan)

    flag_docs_to_remove = ['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_20']
    df = df.drop(flag_docs_to_remove, axis=1)

    df['DAYS_BIRTH'] = df['DAYS_BIRTH'] * -1 / 365

    df['OBS_30_CNT_SOCIAL_CIRCLE'] = df['OBS_30_CNT_SOCIAL_CIRCLE'].apply(lambda x: np.nan if x > 30 else x)
    df['OBS_60_CNT_SOCIAL_CIRCLE'] = df['OBS_60_CNT_SOCIAL_CIRCLE'].apply(lambda x: np.nan if x > 30 else x)

    categorical_columns = df.select_dtypes(include=['object']).columns
    df[categorical_columns] = df[categorical_columns].fillna('XNA')

    df['REGION_RATING_CLIENT'] = df['REGION_RATING_CLIENT'].astype('object')
    df['REGION_RATING_CLIENT_W_CITY'] = df['REGION_RATING_CLIENT_W_CITY'].astype('object')

    new_features = {
        'CREDIT_INCOME_PERCENT': df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL'],
        'ANNUITY_INCOME_PERCENT': df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL'],
        'CREDIT_TERM': df['AMT_ANNUITY'] / df['AMT_CREDIT'],
        'DAYS_EMPLOYED_PERCENT': df['DAYS_EMPLOYED'] / df['DAYS_BIRTH'],
        'INCOME_CREDIT_PERCENTAGE': df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT'],
        'INCOME_PER_PERSON': df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS'],
        'ANNUITY_LENGTH': df['AMT_CREDIT'] / df['AMT_ANNUITY'],
        'CHILDREN_RATIO': df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS'],
        'CREDIT_TO_GOODS_RATIO': df['AMT_CREDIT'] / df['AMT_GOODS_PRICE'],
        'INC_PER_CHLD': df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN']),
        'SOURCES_PROD': df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3'],
        'CAR_TO_BIRTH_RATIO': df['OWN_CAR_AGE'] / df['DAYS_BIRTH'],
        'CAR_TO_EMPLOY_RATIO': df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED'],
        'PHONE_TO_BIRTH_RATIO': df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH'],
        'PHONE_TO_EMPLOY_RATIO': df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
    }

    new_features.update({
        'CREDIT_INCOME_RATIO': df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL'],
        'CREDIT_ANNUITY_RATIO': df['AMT_CREDIT'] / df['AMT_ANNUITY'],
        'INCOME_ANNUITY_DIFF': df['AMT_INCOME_TOTAL'] - df['AMT_ANNUITY'],
        'CREDIT_GOODS_DIFF': df['AMT_CREDIT'] - df['AMT_GOODS_PRICE'],
        'GOODS_INCOME_RATIO': df['AMT_GOODS_PRICE'] / df['AMT_INCOME_TOTAL'],
        'INCOME_EXT_RATIO': df['AMT_INCOME_TOTAL'] / df['EXT_SOURCE_3'],
        'CREDIT_EXT_RATIO': df['AMT_CREDIT'] / df['EXT_SOURCE_3'],
        'AGE_EMPLOYED_DIFF': df['DAYS_BIRTH'] - df['DAYS_EMPLOYED'],
        'EMPLOYED_TO_AGE_RATIO': df['DAYS_EMPLOYED'] / df['DAYS_BIRTH'],
        'CAR_EMPLOYED_DIFF': df['OWN_CAR_AGE'] - df['DAYS_EMPLOYED'],
        'CAR_AGE_DIFF': df['DAYS_BIRTH'] - df['OWN_CAR_AGE'],
        'CAR_AGE_RATIO': df['OWN_CAR_AGE'] / df['DAYS_BIRTH'],
        'FLAG_CONTACTS_SUM': df['FLAG_MOBIL'] + df['FLAG_EMP_PHONE'] + df['FLAG_WORK_PHONE'] + df['FLAG_CONT_MOBILE'] + df['FLAG_PHONE'] + df['FLAG_EMAIL'],
        'HOUR_PROCESS_CREDIT_MUL': df['AMT_CREDIT'] * df['HOUR_APPR_PROCESS_START'],
        'CNT_NON_CHILDREN': df['CNT_FAM_MEMBERS'] - df['CNT_CHILDREN'],
        'CHILDREN_INCOME_RATIO': df['CNT_CHILDREN'] / df['AMT_INCOME_TOTAL'],
        'PER_CAPITA_INCOME': df['AMT_INCOME_TOTAL'] / (df['CNT_FAM_MEMBERS'] + 1),
        'REGIONS_RATING_INCOME_MUL': (df['REGION_RATING_CLIENT'] + df['REGION_RATING_CLIENT_W_CITY']) * df['AMT_INCOME_TOTAL'] / 2
    })

    new_features.update({
        'REGION_RATING_MAX': df[['REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY']].max(axis=1),
        'REGION_RATING_MIN': df[['REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY']].min(axis=1),
        'REGION_RATING_MEAN': (df['REGION_RATING_CLIENT'] + df['REGION_RATING_CLIENT_W_CITY']) / 2,
        'REGION_RATING_MUL': df['REGION_RATING_CLIENT'] * df['REGION_RATING_CLIENT_W_CITY'],
        'FLAG_REGIONS': df['REG_REGION_NOT_LIVE_REGION'] + df['REG_REGION_NOT_WORK_REGION'] + df['LIVE_REGION_NOT_WORK_REGION'] + df['REG_CITY_NOT_LIVE_CITY'] + df['REG_CITY_NOT_WORK_CITY'] + df['LIVE_CITY_NOT_WORK_CITY'],
        'EXT_SOURCE_MEAN': df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1),
        'EXT_SOURCE_MUL': df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3'],
        'EXT_SOURCE_MAX': df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1),
        'EXT_SOURCE_MIN': df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1),
        'EXT_SOURCE_VAR': df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].var(axis=1),
        'WEIGHTED_EXT_SOURCE': df['EXT_SOURCE_1'] * 2 + df['EXT_SOURCE_2'] * 3 + df['EXT_SOURCE_3'] * 4
    })

    apartment_avg_columns = [col for col in df.columns if 'AVG' in col]
    apartment_mode_columns = [col for col in df.columns if 'MODE' in col]
    apartment_medi_columns = [col for col in df.columns if 'MEDI' in col]
    
    for col in apartment_mode_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    new_features.update({
        'APARTMENTS_SUM_AVG': df[apartment_avg_columns].sum(axis=1),
        'APARTMENTS_SUM_MODE': df[apartment_mode_columns].sum(axis=1),
        'APARTMENTS_SUM_MEDI': df[apartment_medi_columns].sum(axis=1),
        'INCOME_APARTMENT_AVG_MUL': df[apartment_avg_columns].sum(axis=1) * df['AMT_INCOME_TOTAL'],
        'INCOME_APARTMENT_MODE_MUL': df[apartment_mode_columns].sum(axis=1) * df['AMT_INCOME_TOTAL'],
        'INCOME_APARTMENT_MEDI_MUL': df[apartment_medi_columns].sum(axis=1) * df['AMT_INCOME_TOTAL']
    })

    new_features.update({
        'OBS_30_60_SUM': df['OBS_30_CNT_SOCIAL_CIRCLE'] + df['OBS_60_CNT_SOCIAL_CIRCLE'],
        'DEF_30_60_SUM': df['DEF_30_CNT_SOCIAL_CIRCLE'] + df['DEF_60_CNT_SOCIAL_CIRCLE'],
        'OBS_DEF_30_MUL': df['OBS_30_CNT_SOCIAL_CIRCLE'] * df['DEF_30_CNT_SOCIAL_CIRCLE'],
        'OBS_DEF_60_MUL': df['OBS_60_CNT_SOCIAL_CIRCLE'] * df['DEF_60_CNT_SOCIAL_CIRCLE'],
        'SUM_OBS_DEF_ALL': df['OBS_30_CNT_SOCIAL_CIRCLE'] + df['DEF_30_CNT_SOCIAL_CIRCLE'] + df['OBS_60_CNT_SOCIAL_CIRCLE'] + df['DEF_60_CNT_SOCIAL_CIRCLE'],
        'OBS_30_CREDIT_RATIO': df['AMT_CREDIT'] / (df['OBS_30_CNT_SOCIAL_CIRCLE'] + 0.00001),
        'OBS_60_CREDIT_RATIO': df['AMT_CREDIT'] / (df['OBS_60_CNT_SOCIAL_CIRCLE'] + 0.00001),
        'DEF_30_CREDIT_RATIO': df['AMT_CREDIT'] / (df['DEF_30_CNT_SOCIAL_CIRCLE'] + 0.00001),
        'DEF_60_CREDIT_RATIO': df['AMT_CREDIT'] / (df['DEF_60_CNT_SOCIAL_CIRCLE'] + 0.00001)
    })

    flag_doc_columns = [col for col in df.columns if col.startswith('FLAG_DOCUMENT') and col not in flag_docs_to_remove]
    new_features['SUM_FLAGS_DOCUMENTS'] = df[flag_doc_columns].sum(axis=1)

    new_features.update({
        'DAYS_DETAILS_CHANGE_MUL': df['DAYS_LAST_PHONE_CHANGE'] * df['DAYS_REGISTRATION'] * df['DAYS_ID_PUBLISH'],
        'DAYS_DETAILS_CHANGE_SUM': df['DAYS_LAST_PHONE_CHANGE'] + df['DAYS_REGISTRATION'] + df['DAYS_ID_PUBLISH']
    })

    enquiry_columns = [col for col in df.columns if col.startswith('AMT_REQ_CREDIT_BUREAU')]
    new_features.update({
        'AMT_ENQ_SUM': df[enquiry_columns].sum(axis=1),
        'ENQ_CREDIT_RATIO': df[enquiry_columns].sum(axis=1) / df['AMT_CREDIT']
    })

    df = pd.concat([df, pd.DataFrame(new_features)], axis=1)

    df['MISSING_VALS_TOTAL_APP'] = df.isna().sum(axis=1)

    if is_train:
        y = df['TARGET']
        df = df.drop('TARGET', axis=1)
        return df, y
    else:
        return df


 ## Impute Missing Values

 Two functions handle missing value imputation:

1. `impute_ext_source`:
   - Uses LightGBM to predict missing values in EXT_SOURCE columns.
   - Encodes categorical variables before model training.

2. `knn_impute_target`:
   - Uses K-Nearest Neighbors to impute TARGET mean for each application.
   - This helps in creating a feature based on similar applications' outcomes.

In [103]:
def impute_ext_source(train_df, test_df):
    """Impute missing values in EXT_SOURCE columns"""
    le = LabelEncoder()
    
    for ext_col in ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']:
        columns_to_drop = ['SK_ID_CURR', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
        if 'TARGET' in train_df.columns:
            columns_to_drop.append('TARGET')
        
        features = [col for col in train_df.columns if col not in columns_to_drop]
        
        for feature in features:
            if train_df[feature].dtype == 'object':
                # Combine train and test data for encoding
                combined = pd.concat([train_df[feature], test_df[feature]], axis=0)
                le.fit(combined.astype(str))
                train_df[feature] = le.transform(train_df[feature].astype(str))
                test_df[feature] = le.transform(test_df[feature].astype(str))
        
        X_model = train_df[~train_df[ext_col].isna()][features]
        y_model = train_df[~train_df[ext_col].isna()][ext_col]
        X_train_missing = train_df[train_df[ext_col].isna()][features]
        X_test_missing = test_df[test_df[ext_col].isna()][features]

        model = LGBMRegressor(n_estimators=1000, learning_rate=0.05, num_leaves=32, n_jobs=-1, random_state=42)
        model.fit(X_model, y_model)

        train_df.loc[train_df[ext_col].isna(), ext_col] = model.predict(X_train_missing)
        test_df.loc[test_df[ext_col].isna(), ext_col] = model.predict(X_test_missing)

    return train_df, test_df

def knn_impute_target(train_df, test_df):
    """Impute TARGET mean of 500 nearest neighbors"""
    features_for_knn = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']
    
    if 'TARGET' not in train_df.columns:
        print("Warning: 'TARGET' column not found in training data. Skipping KNN imputation.")
        return train_df, test_df
    
    knn = KNeighborsClassifier(n_neighbors=500, n_jobs=-1)
    train_features = train_df[features_for_knn].fillna(-999)
    test_features = test_df[features_for_knn].fillna(-999)
    
    knn.fit(train_features, train_df['TARGET'])
    
    train_neighbors = knn.kneighbors(train_features)[1]
    test_neighbors = knn.kneighbors(test_features)[1]
    
    train_df['TARGET_NEIGHBORS_MEAN'] = [train_df['TARGET'].iloc[indices].mean() for indices in train_neighbors]
    test_df['TARGET_NEIGHBORS_MEAN'] = [train_df['TARGET'].iloc[indices].mean() for indices in test_neighbors]
    
    return train_df, test_df


 ## Process Bureau Data

 The `process_bureau` function handles credit bureau data:

1. Reads and reduces memory usage of the bureau dataframe.
2. Performs feature engineering on bureau data.
3. Processes bureau_balance data:
   - Converts STATUS to numeric.
   - Creates weighted status features.
   - Calculates exponential weighted averages.
4. Aggregates bureau_balance data.
5. Aggregates bureau data, creating summary statistics for each SK_ID_CURR.


In [104]:
def process_bureau(file_path):
    """Process bureau.csv and bureau_balance.csv"""
    bureau = read_csv(file_path)
    bureau = reduce_mem_usage(bureau)

    bureau = bureau.fillna(0)

    bureau['CREDIT_DURATION'] = bureau['DAYS_CREDIT'] - bureau['DAYS_CREDIT_ENDDATE']
    
    new_features = {
        'FLAG_OVERDUE_RECENT': (bureau['CREDIT_DAY_OVERDUE'] > 0).astype(int),
        'MAX_AMT_OVERDUE_DURATION_RATIO': bureau['AMT_CREDIT_MAX_OVERDUE'] / (bureau['CREDIT_DURATION'] + 0.00001),
        'CURRENT_AMT_OVERDUE_DURATION_RATIO': bureau['AMT_CREDIT_SUM_OVERDUE'] / (bureau['CREDIT_DURATION'] + 0.00001),
        'AMT_OVERDUE_DURATION_LEFT_RATIO': bureau['AMT_CREDIT_SUM_OVERDUE'] / (bureau['DAYS_CREDIT_ENDDATE'] + 0.00001),
        'CNT_PROLONGED_MAX_OVERDUE_MUL': bureau['CNT_CREDIT_PROLONG'] * bureau['AMT_CREDIT_MAX_OVERDUE'],
        'CNT_PROLONGED_DURATION_RATIO': bureau['CNT_CREDIT_PROLONG'] / (bureau['CREDIT_DURATION'] + 0.00001),
        'CURRENT_DEBT_TO_CREDIT_RATIO': bureau['AMT_CREDIT_SUM_DEBT'] / (bureau['AMT_CREDIT_SUM'] + 0.00001),
        'CURRENT_CREDIT_DEBT_DIFF': bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT'],
        'AMT_ANNUITY_CREDIT_RATIO': bureau['AMT_ANNUITY'] / (bureau['AMT_CREDIT_SUM'] + 0.00001),
        'CREDIT_ENDDATE_UPDATE_DIFF': abs(bureau['DAYS_CREDIT_UPDATE'] - bureau['DAYS_CREDIT_ENDDATE'])
    }

    bureau = pd.concat([bureau, pd.DataFrame(new_features)], axis=1)

    bb_file = file_path.replace('bureau.csv', 'bureau_balance.csv')
    bb = read_csv(bb_file)
    bb = reduce_mem_usage(bb)

    status_map = {'C': 0, '0': 1, '1': 2, '2': 3, 'X': 4, '3': 5, '4': 6, '5': 7}
    bb['STATUS'] = bb['STATUS'].map(status_map)

    bb['MONTHS_BALANCE'] = abs(bb['MONTHS_BALANCE'])
    bb['WEIGHTED_STATUS'] = bb['STATUS'] / (bb['MONTHS_BALANCE'] + 1)

    bb = bb.sort_values(['SK_ID_BUREAU', 'MONTHS_BALANCE'], ascending=[True, False])
    bb['EXP_WEIGHTED_STATUS'] = bb.groupby('SK_ID_BUREAU')['WEIGHTED_STATUS'].transform(lambda x: x.ewm(alpha=0.8).mean())
    bb['EXP_ENCODED_STATUS'] = bb.groupby('SK_ID_BUREAU')['STATUS'].transform(lambda x: x.ewm(alpha=0.8).mean())

    bb_aggregations = {
        'MONTHS_BALANCE': ['max', 'mean'],
        'STATUS': ['mean', 'max', 'min'],
        'WEIGHTED_STATUS': ['mean', 'max', 'min'],
        'EXP_WEIGHTED_STATUS': ['last'],
        'EXP_ENCODED_STATUS': ['last']
    }
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([f'BB_{e[0]}_{e[1].upper()}' for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')

    aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
        'CNT_CREDIT_PROLONG': ['sum', 'mean', 'max'],
        'AMT_CREDIT_SUM': ['sum', 'mean', 'max'],
        'AMT_CREDIT_SUM_DEBT': ['sum', 'mean', 'max'],
        'AMT_CREDIT_SUM_LIMIT': ['sum', 'mean', 'max'],
        'AMT_CREDIT_SUM_OVERDUE': ['sum', 'mean', 'max'],
        'DAYS_CREDIT_UPDATE': ['mean', 'min', 'max'],
        'AMT_ANNUITY': ['sum', 'mean', 'max'],
        'CREDIT_DURATION': ['mean', 'max'],
        'FLAG_OVERDUE_RECENT': ['sum', 'mean'],
        'MAX_AMT_OVERDUE_DURATION_RATIO': ['mean', 'max'],
        'CURRENT_AMT_OVERDUE_DURATION_RATIO': ['mean', 'max'],
        'AMT_OVERDUE_DURATION_LEFT_RATIO': ['mean', 'max'],
        'CNT_PROLONGED_MAX_OVERDUE_MUL': ['mean', 'max'],
        'CNT_PROLONGED_DURATION_RATIO': ['mean', 'max'],
        'CURRENT_DEBT_TO_CREDIT_RATIO': ['mean', 'max'],
        'CURRENT_CREDIT_DEBT_DIFF': ['mean', 'max'],
        'AMT_ANNUITY_CREDIT_RATIO': ['mean', 'max'],
        'CREDIT_ENDDATE_UPDATE_DIFF': ['mean', 'max'],
    }
    bureau_agg = bureau.groupby('SK_ID_CURR').agg(aggregations)
    bureau_agg.columns = pd.Index([f'BURO_{e[0]}_{e[1].upper()}' for e in bureau_agg.columns.tolist()])
    return bureau_agg


 ## Process Previous Application Data

 The `process_previous_application` function handles previous application data:

1. Reads and reduces memory usage of the previous application dataframe.
2. Handles erroneous values.
3. Encodes categorical variables.
4. Performs extensive feature engineering.
5. Aggregates data, creating summary statistics for each SK_ID_CURR.


In [105]:
def process_previous_application(file_path):
    """Process previous_application.csv"""
    prev = read_csv(file_path)
    prev = reduce_mem_usage(prev)

    prev['DAYS_FIRST_DRAWING'] = prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan)
    prev['DAYS_FIRST_DUE'] = prev['DAYS_FIRST_DUE'].replace(365243, np.nan)
    prev['DAYS_LAST_DUE_1ST_VERSION'] = prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan)
    prev['DAYS_LAST_DUE'] = prev['DAYS_LAST_DUE'].replace(365243, np.nan)
    prev['DAYS_TERMINATION'] = prev['DAYS_TERMINATION'].replace(365243, np.nan)
    prev['SELLERPLACE_AREA'] = prev['SELLERPLACE_AREA'].replace(4000000, np.nan)

    le = LabelEncoder()
    categorical_columns = prev.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        prev[col] = le.fit_transform(prev[col].astype(str))

    new_features = {
        'MISSING_VALUES_TOTAL_PREV': prev.isnull().sum(axis=1),
        'AMT_DECLINED': prev['AMT_APPLICATION'] - prev['AMT_CREDIT'],
        'AMT_CREDIT_GOODS_RATIO': prev['AMT_CREDIT'] / (prev['AMT_GOODS_PRICE'] + 0.00001),
        'AMT_CREDIT_GOODS_DIFF': prev['AMT_CREDIT'] - prev['AMT_GOODS_PRICE'],
        'AMT_CREDIT_APPLICATION_RATIO': prev['AMT_APPLICATION'] / (prev['AMT_CREDIT'] + 0.00001),
        'CREDIT_DOWNPAYMENT_RATIO': prev['AMT_DOWN_PAYMENT'] / (prev['AMT_CREDIT'] + 0.00001),
        'GOOD_DOWNPAYMET_RATIO': prev['AMT_DOWN_PAYMENT'] / (prev['AMT_GOODS_PRICE'] + 0.00001),
        'INTEREST_DOWNPAYMENT': prev['RATE_DOWN_PAYMENT'] * prev['AMT_DOWN_PAYMENT'],
        'INTEREST_CREDIT': prev['AMT_CREDIT'] * prev['RATE_INTEREST_PRIMARY'],
        'INTEREST_CREDIT_PRIVILEGED': prev['AMT_CREDIT'] * prev['RATE_INTEREST_PRIVILEGED'],
        'APPLICATION_AMT_TO_DECISION_RATIO': prev['AMT_APPLICATION'] / (prev['DAYS_DECISION'].abs() + 0.00001),
        'AMT_APPLICATION_TO_SELLERPLACE_AREA': prev['AMT_APPLICATION'] / (prev['SELLERPLACE_AREA'] + 0.00001),
        'ANNUITY': prev['AMT_CREDIT'] / (prev['CNT_PAYMENT'] + 0.00001),
        'ANNUITY_GOODS': prev['AMT_GOODS_PRICE'] / (prev['CNT_PAYMENT'] + 0.00001),
        'DAYS_FIRST_LAST_DUE_DIFF': prev['DAYS_LAST_DUE'] - prev['DAYS_FIRST_DUE'],
        'AMT_CREDIT_HOUR_PROCESS_START': prev['AMT_CREDIT'] * prev['HOUR_APPR_PROCESS_START'],
        'AMT_CREDIT_NFLAG_LAST_APPL_DAY': prev['AMT_CREDIT'] * prev['NFLAG_LAST_APPL_IN_DAY'],
        'AMT_CREDIT_YIELD_GROUP': prev['AMT_CREDIT'] * prev['NAME_YIELD_GROUP'],
        'AMT_INTEREST': prev['CNT_PAYMENT'] * prev['AMT_ANNUITY'] - prev['AMT_CREDIT'],
        'INTEREST_SHARE': (prev['CNT_PAYMENT'] * prev['AMT_ANNUITY'] - prev['AMT_CREDIT']) / (prev['AMT_CREDIT'] + 0.00001),
        'INTEREST_RATE': 2 * 12 * (prev['CNT_PAYMENT'] * prev['AMT_ANNUITY'] - prev['AMT_CREDIT']) / (prev['AMT_CREDIT'] * (prev['CNT_PAYMENT'] + 1))
    }

    prev = pd.concat([prev, pd.DataFrame(new_features)], axis=1)

    aggregations = {
        'SK_ID_CURR': ['count'],
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        'DAYS_FIRST_DRAWING': ['min', 'max', 'mean'],
        'DAYS_FIRST_DUE': ['min', 'max', 'mean'],
        'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
        'DAYS_LAST_DUE': ['min', 'max', 'mean'],
        'DAYS_TERMINATION': ['min', 'max', 'mean'],
        'NFLAG_INSURED_ON_APPROVAL': ['mean'],
        'MISSING_VALUES_TOTAL_PREV': ['sum', 'mean'],
        'AMT_DECLINED': ['min', 'max', 'mean'],
        'AMT_CREDIT_GOODS_RATIO': ['min', 'max', 'mean'],
        'AMT_CREDIT_GOODS_DIFF': ['min', 'max', 'mean'],
        'AMT_CREDIT_APPLICATION_RATIO': ['min', 'max', 'mean'],
        'CREDIT_DOWNPAYMENT_RATIO': ['min', 'max', 'mean'],
        'GOOD_DOWNPAYMET_RATIO': ['min', 'max', 'mean'],
        'INTEREST_DOWNPAYMENT': ['min', 'max', 'mean'],
        'INTEREST_CREDIT': ['min', 'max', 'mean'],
        'INTEREST_CREDIT_PRIVILEGED': ['min', 'max', 'mean'],
        'APPLICATION_AMT_TO_DECISION_RATIO': ['min', 'max', 'mean'],
        'AMT_APPLICATION_TO_SELLERPLACE_AREA': ['min', 'max', 'mean'],
        'ANNUITY': ['min', 'max', 'mean'],
        'ANNUITY_GOODS': ['min', 'max', 'mean'],
        'DAYS_FIRST_LAST_DUE_DIFF': ['min', 'max', 'mean'],
        'AMT_CREDIT_HOUR_PROCESS_START': ['min', 'max', 'mean'],
        'AMT_CREDIT_NFLAG_LAST_APPL_DAY': ['min', 'max', 'mean'],
        'AMT_CREDIT_YIELD_GROUP': ['min', 'max', 'mean'],
        'AMT_INTEREST': ['min', 'max', 'mean'],
        'INTEREST_SHARE': ['min', 'max', 'mean'],
        'INTEREST_RATE': ['min', 'max', 'mean']
    }
    prev_agg = prev.groupby('SK_ID_CURR').agg(aggregations)
    prev_agg.columns = pd.Index([f'PREV_{e[0]}_{e[1].upper()}' for e in prev_agg.columns.tolist()])
    return prev_agg

 ## Process POS CASH Balance Data

 The `process_pos_cash` function handles POS CASH balance data:

1. Reads and reduces memory usage of the POS CASH balance dataframe.
2. Performs feature engineering, including exponential moving averages.
3. Aggregates data, creating summary statistics for each SK_ID_CURR.
4. Creates and aggregates dummy variables for NAME_CONTRACT_STATUS.

In [106]:
def process_pos_cash(file_path):
    """Process POS_CASH_balance.csv"""
    pos = read_csv(file_path)
    pos = reduce_mem_usage(pos)

    pos['MONTHS_BALANCE'] = pos['MONTHS_BALANCE'].abs()

    pos = pos.sort_values(['SK_ID_PREV', 'MONTHS_BALANCE'], ascending=[True, False])

    new_features = {
        'SK_DPD_RATIO': pos['SK_DPD'] / (pos['SK_DPD_DEF'] + 0.00001),
        'TOTAL_TERM': pos['CNT_INSTALMENT'] + pos['CNT_INSTALMENT_FUTURE']
    }

    for col in ['CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE']:
        new_features[f'EXP_{col}'] = pos.groupby('SK_ID_PREV')[col].transform(lambda x: x.ewm(alpha=0.6).mean())

    new_features['EXP_POS_TOTAL_TERM'] = new_features['EXP_CNT_INSTALMENT'] + new_features['EXP_CNT_INSTALMENT_FUTURE']

    pos = pd.concat([pos, pd.DataFrame(new_features)], axis=1)

    aggregations = {
        'MONTHS_BALANCE': ['max'],
        'CNT_INSTALMENT': ['mean', 'max', 'min'],
        'CNT_INSTALMENT_FUTURE': ['mean', 'max', 'min'],
        'SK_DPD': ['max', 'sum'],
        'SK_DPD_DEF': ['max', 'sum'],
        'SK_DPD_RATIO': ['mean', 'max'],
        'TOTAL_TERM': ['mean', 'max', 'last'],
        'EXP_CNT_INSTALMENT': ['last'],
        'EXP_CNT_INSTALMENT_FUTURE': ['last'],
        'EXP_POS_TOTAL_TERM': ['mean']
    }

    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index([f'POS_{e[0]}_{e[1].upper()}' for e in pos_agg.columns.tolist()])

    pos_dummies = pd.get_dummies(pos['NAME_CONTRACT_STATUS'], prefix='POS_STATUS')
    
    pos = pd.concat([pos, pos_dummies], axis=1)
    
    dummy_agg = pos.groupby('SK_ID_CURR')[pos_dummies.columns].mean()
    
    pos_agg = pd.concat([pos_agg, dummy_agg], axis=1)

    return pos_agg


 ## Process Installments Payments Data

 The `process_installments_payments` function handles installments payments data:

1. Reads and reduces memory usage of the installments payments dataframe.
2. Performs feature engineering related to payment behavior.
3. Aggregates data, creating summary statistics for each SK_ID_CURR.

In [107]:
# %%
def process_installments_payments(file_path):
    """Process installments_payments.csv"""
    ins = read_csv(file_path)
    ins = reduce_mem_usage(ins)

    # Feature engineering
    new_features = {
        'MISSING_VALS_TOTAL_CC': ins.isnull().sum(axis=1),
        'PAYMENT_PERC': ins['AMT_PAYMENT'] / (ins['AMT_INSTALMENT'] + 0.00001),
        'PAYMENT_DIFF': ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT'],
        'DPD': (ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']).apply(lambda x: max(x, 0)),
        'DBD': (ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']).apply(lambda x: max(x, 0))
    }

    ins = pd.concat([ins, pd.DataFrame(new_features)], axis=1)

    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum'],
        'MISSING_VALS_TOTAL_CC': ['sum']
    }
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INS_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    return ins_agg


 ## Process Credit Card Balance Data

 The `process_credit_card_balance` function handles credit card balance data:

1. Reads and reduces memory usage of the credit card balance dataframe.
2. Handles erroneous values.
3. Performs feature engineering related to credit card usage and payments.
4. Calculates exponential weighted moving averages for several features.
5. Aggregates data, creating summary statistics for each SK_ID_CURR.

In [108]:
def process_credit_card_balance(file_path):
    """Process credit_card_balance.csv"""
    cc = read_csv(file_path)
    cc = reduce_mem_usage(cc)

    cc.loc[cc['AMT_PAYMENT_CURRENT'] > 4000000, 'AMT_PAYMENT_CURRENT'] = np.nan

    new_features = {
        'MISSING_VALS_TOTAL_CC': cc.isnull().sum(axis=1),
        'AMT_DRAWING_SUM': cc['AMT_DRAWINGS_ATM_CURRENT'] + cc['AMT_DRAWINGS_CURRENT'] + cc['AMT_DRAWINGS_OTHER_CURRENT'] + cc['AMT_DRAWINGS_POS_CURRENT'],
        'BALANCE_LIMIT_RATIO': cc['AMT_BALANCE'] / (cc['AMT_CREDIT_LIMIT_ACTUAL'] + 0.00001),
        'CNT_DRAWING_SUM': cc['CNT_DRAWINGS_ATM_CURRENT'] + cc['CNT_DRAWINGS_CURRENT'] + cc['CNT_DRAWINGS_OTHER_CURRENT'] + cc['CNT_DRAWINGS_POS_CURRENT'] + cc['CNT_INSTALMENT_MATURE_CUM'],
        'MIN_PAYMENT_RATIO': cc['AMT_PAYMENT_CURRENT'] / (cc['AMT_INST_MIN_REGULARITY'] + 0.00001),
        'PAYMENT_MIN_DIFF': cc['AMT_PAYMENT_CURRENT'] - cc['AMT_INST_MIN_REGULARITY'],
        'MIN_PAYMENT_TOTAL_RATIO': cc['AMT_PAYMENT_TOTAL_CURRENT'] / (cc['AMT_INST_MIN_REGULARITY'] + 0.00001),
        'AMT_INTEREST_RECEIVABLE': cc['AMT_TOTAL_RECEIVABLE'] - cc['AMT_RECEIVABLE_PRINCIPAL'],
        'SK_DPD_RATIO': cc['SK_DPD'] / (cc['SK_DPD_DEF'] + 0.00001)
    }

    cc = pd.concat([cc, pd.DataFrame(new_features)], axis=1)

    for col in ['AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 
                'AMT_DRAWING_SUM', 'BALANCE_LIMIT_RATIO', 'CNT_DRAWING_SUM', 'MIN_PAYMENT_RATIO', 'PAYMENT_MIN_DIFF', 
                'MIN_PAYMENT_TOTAL_RATIO', 'AMT_INTEREST_RECEIVABLE', 'SK_DPD_RATIO']:
        cc[f'EXP_{col}'] = cc.groupby('SK_ID_PREV')[col].transform(lambda x: x.ewm(alpha=0.7).mean())

    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'AMT_BALANCE': ['max', 'mean'],
        'AMT_CREDIT_LIMIT_ACTUAL': ['max', 'mean'],
        'AMT_DRAWINGS_ATM_CURRENT': ['max', 'mean'],
        'AMT_DRAWINGS_CURRENT': ['max', 'mean'],
        'AMT_DRAWINGS_OTHER_CURRENT': ['max', 'mean'],
        'AMT_DRAWINGS_POS_CURRENT': ['max', 'mean'],
        'AMT_INST_MIN_REGULARITY': ['max', 'mean'],
        'AMT_PAYMENT_CURRENT': ['max', 'mean'],
        'AMT_PAYMENT_TOTAL_CURRENT': ['max', 'mean'],
        'AMT_RECEIVABLE_PRINCIPAL': ['max', 'mean'],
        'AMT_RECIVABLE': ['max', 'mean'],
        'AMT_TOTAL_RECEIVABLE': ['max', 'mean'],
        'CNT_DRAWINGS_ATM_CURRENT': ['max', 'mean'],
        'CNT_DRAWINGS_CURRENT': ['max', 'mean'],
        'CNT_DRAWINGS_OTHER_CURRENT': ['max', 'mean'],
        'CNT_DRAWINGS_POS_CURRENT': ['max', 'mean'],
        'CNT_INSTALMENT_MATURE_CUM': ['max', 'mean'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean'],
        'AMT_DRAWING_SUM': ['max', 'mean'],
        'BALANCE_LIMIT_RATIO': ['max', 'mean'],
        'CNT_DRAWING_SUM': ['max', 'mean'],
        'MIN_PAYMENT_RATIO': ['min', 'mean'],
        'PAYMENT_MIN_DIFF': ['min', 'mean'],
        'MIN_PAYMENT_TOTAL_RATIO': ['min', 'mean'],
        'AMT_INTEREST_RECEIVABLE': ['min', 'mean'],
        'SK_DPD_RATIO': ['max', 'mean'],
        'MISSING_VALS_TOTAL_CC': ['sum'],
    }

    for col in cc.columns:
        if col.startswith('EXP_'):
            aggregations[col] = ['last']

    cc_agg = cc.groupby('SK_ID_CURR').agg(aggregations)
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    return cc_agg


 ## Main Processing Function

 The `main` function orchestrates the entire data processing pipeline:

1. Processes application data for both train and test sets.
2. Imputes missing values in EXT_SOURCE columns and performs KNN-based TARGET imputation.
3. Processes and joins data from all other datasets (bureau, previous application, POS CASH balance, installments payments, credit card balance).
4. Handles any remaining missing values by filling with mean values.
5. Saves the processed datasets as CSV files.

In [109]:
def main():
    print("Processing application_train.csv and application_test.csv...")
    train_df, y = process_application(APPLICATION_TRAIN, is_train=True)
    test_df = process_application(APPLICATION_TEST, is_train=False)

    print("Imputing missing values in EXT_SOURCE columns...")
    train_df, test_df = impute_ext_source(train_df, test_df)

    print("Imputing TARGET mean of 500 nearest neighbors...")
    train_df, test_df = knn_impute_target(train_df, test_df)

    print("Processing bureau.csv and bureau_balance.csv...")
    bureau_agg = process_bureau(BUREAU)
    train_df = train_df.join(bureau_agg, how='left', on='SK_ID_CURR')
    test_df = test_df.join(bureau_agg, how='left', on='SK_ID_CURR')
    del bureau_agg
    gc.collect()

    print("Processing previous_application.csv...")
    prev_agg = process_previous_application(PREVIOUS_APPLICATION)
    train_df = train_df.join(prev_agg, how='left', on='SK_ID_CURR')
    test_df = test_df.join(prev_agg, how='left', on='SK_ID_CURR')
    del prev_agg
    gc.collect()

    print("Processing POS_CASH_balance.csv...")
    pos_agg = process_pos_cash(POS_CASH)
    train_df = train_df.join(pos_agg, how='left', on='SK_ID_CURR')
    test_df = test_df.join(pos_agg, how='left', on='SK_ID_CURR')
    del pos_agg
    gc.collect()

    print("Processing installments_payments.csv...")
    ins_agg = process_installments_payments(INSTALLMENTS_PAYMENTS)
    train_df = train_df.join(ins_agg, how='left', on='SK_ID_CURR')
    test_df = test_df.join(ins_agg, how='left', on='SK_ID_CURR')
    del ins_agg
    gc.collect()

    print("Processing credit_card_balance.csv...")
    cc_agg = process_credit_card_balance(CREDIT_CARD_BALANCE)
    train_df = train_df.join(cc_agg, how='left', on='SK_ID_CURR')
    test_df = test_df.join(cc_agg, how='left', on='SK_ID_CURR')
    del cc_agg
    gc.collect()

    print("Final preprocessing steps...")
    # Handle remaining missing values
    train_df = train_df.fillna(train_df.mean())
    test_df = test_df.fillna(test_df.mean())

    print("Saving processed datasets...")
    train_df.to_csv(PROCESSED_TRAIN, index=False)
    test_df.to_csv(PROCESSED_TEST, index=False)
    train_target = pd.DataFrame({'SK_ID_CURR': train_df['SK_ID_CURR'], 'TARGET': y})
    train_target.to_csv(TRAIN_TARGET, index=False)

    print("Processing complete!")


 ## Run the Processing Pipeline

In [110]:
if __name__ == "__main__":
    main()


Processing application_train.csv and application_test.csv...
Imputing missing values in EXT_SOURCE columns...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.151471 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23146
[LightGBM] [Info] Number of data points in the train set: 134133, number of used features: 172
[LightGBM] [Info] Start training from score 0.502129
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.263090 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23281
[LightGBM] [Info] Number of data points in the train set: 306851, number of used features: 172
[LightGBM] [Info] Start training from score 0.514393
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.213098 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23162
[Li

 ## Conclusion



 This notebook has processed the Home Credit Default Risk dataset, including:



 1. Data cleaning and initial feature engineering for each dataset

 2. Imputation of missing values, especially for EXT_SOURCE columns

 3. KNN-based imputation for TARGET-related features

 4. Aggregation of data from multiple related tables

 5. Final imputation of any remaining missing values



 The processed datasets are saved as CSV files for further analysis and modeling.